In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/10/11 00:35:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [6]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [9]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [13]:
df_join = spark.read.parquet('data/report/revenue/total')

In [14]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 05:00:00| 143|              null|                null|            275.12|                    9|
|2020-01-01 05:00:00| 161|              null|                null| 979.1199999999997|                   39|
|2020-01-01 06:00:00|  92|             28.66|                   2|             71.16|                    1|
|2020-01-01 06:00:00|  93|              null|                null|            330.47|                    5|
|2020-01-01 07:00:00|  47|              2.75|                   1|              19.3|                    1|
|2020-01-01 08:00:00| 142|              null|                null|1331.8899999999992|                   54|
|2020-01-01 09:00:00| 138|  

In [15]:
df_zones = spark.read.parquet('zones/')

In [17]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [16]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [18]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 05:00:00| 143|              null|                null|            275.12|                    9|       143|Manhattan| Lincoln Square West| Yellow Zone|
|2020-01-01 05:00:00| 161|              null|                null| 979.1199999999997|                   39|       161|Manhattan|      Midtown Center| Yellow Zone|
|2020-01-01 06:00:00|  92|             28.66|                   2|             71.16|                    1|        92|   Queens|            Flushing|   Boro Zone|
|2020-01-01 06:00:00| 

In [22]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones', mode = 'overwrite')

In [23]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 05:00:00|              null|                null|            275.12|                    9|Manhattan| Yellow Zone|
|2020-01-01 05:00:00|              null|                null| 979.1199999999997|                   39|Manhattan| Yellow Zone|
|2020-01-01 06:00:00|             28.66|                   2|             71.16|                    1|   Queens|   Boro Zone|
|2020-01-01 06:00:00|              null|                null|            330.47|                    5|   Queens|   Boro Zone|
|2020-01-01 07:00:00|              2.75|                   1|              19.3|                    1|    Bronx|   Bor